In [1]:
import numpy as np
import pandas as pd
import os

files = []

for dirname, _, filenames in os.walk('./Raw_data'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))
        
df_list = [pd.read_csv(file) for file in files]
df = pd.concat(df_list)

df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
11681,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001"
11682,259354,iPhone,1,700,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016"
11683,259355,iPhone,1,700,09/23/19 07:39,"220 12th St, San Francisco, CA 94016"
11684,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016"


In [2]:
df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'], errors = 'coerce')
df['Price Each'] = pd.to_numeric(df['Price Each'], errors = 'coerce')

df['Total Cost'] = df['Quantity Ordered']* df['Price Each'] 

conditions = [df['Price Each'] >= 99.99,
             df['Price Each'] < 99.99]

values = ['High Value', 'Low Value']

df['Price Category'] = np.select(conditions, values)

df['Order Date'] = pd.to_datetime(df['Order Date'],format = '%m/%d/%y %H:%M', errors = 'coerce')

df = df[df['Order Date'].isna()== False]

address = df['Purchase Address'].str.rsplit(' ')

df['State'] = address.str[-2]
df['Zipcode'] = address.str[-1]

df.drop(['Purchase Address'], axis = 1, inplace = True)

df.sort_values('Order Date', inplace = True, ignore_index = True)

df

C:\Users\keith\AppData\Local\Temp\ipykernel_18396\1932880102.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['State'] = address.str[-2]
C:\Users\keith\AppData\Local\Temp\ipykernel_18396\1932880102.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Zipcode'] = address.str[-1]
C:\Users\keith\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Total Cost,Price Category,State,Zipcode
0,147268,Wired Headphones,1.0,11.99,2019-01-01 03:07:00,11.99,Low Value,NY,10001
1,148041,USB-C Charging Cable,1.0,11.95,2019-01-01 03:40:00,11.95,Low Value,CA,94016
2,149343,Apple Airpods Headphones,1.0,150.00,2019-01-01 04:56:00,150.00,High Value,NY,10001
3,149964,AAA Batteries (4-pack),1.0,2.99,2019-01-01 05:53:00,2.99,Low Value,TX,75001
4,149350,USB-C Charging Cable,2.0,11.95,2019-01-01 06:03:00,23.90,Low Value,GA,30301
...,...,...,...,...,...,...,...,...,...
185945,304165,AAA Batteries (4-pack),1.0,2.99,2020-01-01 04:13:00,2.99,Low Value,OR,97035
185946,299125,USB-C Charging Cable,1.0,11.95,2020-01-01 04:21:00,11.95,Low Value,NY,10001
185947,305840,Bose SoundSport Headphones,1.0,99.99,2020-01-01 04:54:00,99.99,High Value,CA,94016
185948,300519,Bose SoundSport Headphones,1.0,99.99,2020-01-01 05:13:00,99.99,High Value,NY,10001


In [3]:
df.to_pickle('Cleaned_df')

In [7]:
import plotly.express as px

df = pd.read_pickle('Cleaned_df')


def group_state_products(dataframe, count_or_cost, metric = 'sum'):
    """
    Will group dataframe by state and product, using specified metric to analyse (e.g. sum, mean etc.) 
    on either the 'Quantity Ordered' or 'Total Cost' columns
    
    """
    
    grouped_state = dataframe.groupby(['State', 'Price Category'])[count_or_cost].agg(metric)
    grouped_state.sort_index(level = 0, inplace = True)
    grouped_state = grouped_state.unstack(level = 1)
    
    grouped_state.rename(columns = {'High Value': 'High Value Item Sales', 'Low Value': 'Low Value Item Sales'}, inplace = True)
    
    return grouped_state

state_data = group_state_products(df, 'Quantity Ordered')
state_data.reset_index(inplace=True)
state_data['Total Sales'] = state_data['High Value Item Sales'] + state_data['Low Value Item Sales']
state_geo = "./Raw_data/us-states.json"

map_fig = px.choropleth(state_data, geojson=state_geo, locations='State', color='Total Sales',
                           color_continuous_scale="Viridis",
                           scope="usa",
                          )

map_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

map_fig.show()